# EduAgent20250629

# 跑通批改流程，验证了大题针对性批改策略的可行性
## 政治、物理、数学、地理

思路：先用API请求最强大的模型验证可行性；再迁移到备选的开源模型；最终落实到本地部署模型接管全流程

### 物理 数学：把答案拆分为基本的给分单元，对每个给分单元进行详细评分

![图片描述](image.png)

In [ ]:
import os, io
from pydantic import BaseModel
from typing import List, Optional, Literal
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

from openai import OpenAI

# 加载环境变量
load_dotenv()


client = OpenAI()
question_path = "question.txt"
ground_truth_answer_path = "ground_truth_answer.txt"
explanation_of_ground_truth_answer_path = "explanation.txt"
few_shot_path = "few_shot_example.json"
student_answer_path = "student_answer.txt"

question = pd.read_table(f"{question_path}")
ground_truth_answer = pd.read_table(f"{ground_truth_answer_path}")
explanation_of_ground_truth_answer = pd.read_table(f"{explanation_of_ground_truth_answer_path}")
few_shot = pd.read_table(f"{few_shot_path}")
student_answer = pd.read_table(f"{student_answer_path}")

class PointsEarnedAndWhy(BaseModel):
    points_earned_of_this_equation:int
    why: str

class CorrectionAndExplanation(BaseModel):
    formula1 : PointsEarnedAndWhy
    formula2 : PointsEarnedAndWhy
    formula3 : PointsEarnedAndWhy

# 定义结构化输出的数据模型
class QuestionGrading(BaseModel):
    points_earned_of_this_question: float
    correction_and_explanation: CorrectionAndExplanation

response = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "现在你是一个中学老师，你要负责批改你学生的物理试卷的题目，主任要求你严格按照题目解析与各点评分标准里的踩分点来进行批改得分，并对每一个踩分点进行解释，比如说这个踩分点有对应的公式，得到相应的分数，那个踩分点没有公式或者公式错误，不得分"}, # 角色扮演 背景介绍
            {"type": "input_text", "text": f"题目：{question}"},
            {"type": "input_text", "text": f"标准答案：{ground_truth_answer}"},
            {"type": "input_text", "text": f"题目解析与各点评分标准：{explanation_of_ground_truth_answer}"},
            {"type": "input_text", "text": f"批改示例：{few_shot}"},
            {"type": "input_text", "text": f"学生答案：{student_answer}"},
        ],
    }],
    text_format = QuestionGrading,
)

# print("现在你是一个中学老师，你要负责批改你学生的物理试卷的题目，主任要求你严格按照题目解析与各点评分标准里的踩分点来进行批改得分，并对每一个踩分点进行解释，比如说这个踩分点有对应的公式，得到相应的分数，那个踩分点没有公式或者公式错误，不得分")
# print(f"题目：{question}")
# print(f"标准答案：{ground_truth_answer}")
# print(f"题目解析与各点评分标准：{explanation_of_ground_truth_answer}")
# print(f"批改示例：{few_shot}")
# print(f"学生答案：{student_answer}")
print("第一轮对话",response.output_parsed)


## 政治 历史 地理：高亮标识可能的得分点，大大减轻阅卷老师的视觉负担，把复杂的综合研判过程简化为“接受或拒绝“

### 政治：教材原文优先，用确定性技术方案来匹配 --> difflib 最大公共字符串
### 历史 地理：语义空间更加灵活，可采用自然语义匹配 --> 本地部署embedding模型 

In [24]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

def get_similarity(target_str, compare_str):
    return SequenceMatcher(None, target_str, compare_str).ratio()

target_str = "经济全球化"
compare_set = ["ABCDEFG经济全球化", "经济全球", "经济全球化", "经济的全球化"]

for compare_str in compare_set:
    similarity = get_similarity(target_str, compare_str)
    print(f"{compare_str}: {similarity}")




ABCDEFG经济全球化: 0.5882352941176471
经济全球: 0.8888888888888888
经济全球化: 1.0
经济的全球化: 0.9090909090909091


In [ ]:
import re
import numpy as np
from paddleocr import PaddleOCR
import ollama
import chromadb
from difflib import SequenceMatcher

def get_center_y(box):
    return (box[0][1] + box[3][1]) / 2

def group_by_lines(char_list, y_thresh=15):
    """将字符按Y坐标聚类为多行"""
    lines = []
    for ch in char_list:
        cy = get_center_y(ch['box'])
        matched = False
        for line in lines:
            line_cy = get_center_y(line[0]['box'])
            if abs(cy - line_cy) < y_thresh:
                line.append(ch)
                matched = True
                break
        if not matched:
            lines.append([ch])
    return lines

def build_colored_path_from_lines(line_groups):
    """将多行字符构造成一个连续染色区域（首尾字符拼接）"""
    path_points = []
    for line in line_groups:
        first = line[0]['box']
        last = line[-1]['box']
        tl = first[0]
        bl = first[3]
        tr = last[1]
        br = last[2]
        path_points.extend([tl, tr, br, bl])
    return [list(map(int, pt)) for pt in path_points]

# 初始化 OCR
ocr = PaddleOCR(use_textline_orientation=True, lang='ch')
res = ocr.predict("image_input.png")[0]

texts = res['rec_texts']
scores = res['rec_scores']
boxes = res['rec_polys']

# 构建字符级序列
char_stream = []
for text, score, box in zip(texts, scores, boxes):
    chars = list(text.strip())
    n = len(chars)
    box = np.array(box)
    top_line = np.linspace(box[0], box[1], n + 1)
    bottom_line = np.linspace(box[3], box[2], n + 1)
    for i, ch in enumerate(chars):
        tl = top_line[i]
        tr = top_line[i+1]
        br = bottom_line[i+1]
        bl = bottom_line[i]
        char_box = np.array([tl, tr, br, bl], dtype=int).tolist()
        char_stream.append({'char': ch, 'box': char_box, 'score': score})

print(char_stream[0])

# 构建全文
full_text = ''.join([c['char'] for c in char_stream])

# 切分为句子（排除顿号）
split_pattern = re.compile(r"([，。！？；：])")
segments = []
start_idx = 0
for match in split_pattern.finditer(full_text):
    end_idx = match.end()
    segment_text = full_text[start_idx:end_idx]
    segment_chars = char_stream[start_idx:end_idx]
    line_groups = group_by_lines(segment_chars)
    polygon_path = build_colored_path_from_lines(line_groups)
    segments.append({
        'text': segment_text,
        'box': polygon_path,
        'score': np.mean([c['score'] for c in segment_chars])
    })
    start_idx = end_idx

# 处理结尾残余
if start_idx < len(full_text):
    segment_chars = char_stream[start_idx:]
    segment_text = full_text[start_idx:]
    line_groups = group_by_lines(segment_chars)
    polygon_path = build_colored_path_from_lines(line_groups)
    segments.append({
        'text': segment_text,
        'box': polygon_path,
        'score': np.mean([c['score'] for c in segment_chars])
    })
'''
client = chromadb.Client()
collection = client.create_collection(name="test_2")

# store each document in a vector embedding database
for i, item in enumerate(segments):
  response = ollama.embed(model="mxbai-embed-large", input=item['text'])
  embeddings = response["embeddings"]
  collection.add(
    ids=[str(i)],
    embeddings=embeddings,
    documents=[item['text']],
  )

target = "推动经济全球化朝着更加开放、包容、普惠、平衡的方向发展"
target_embedding = ollama.embed(
  model="mxbai-embed-large",
  input=target
)
query_result = collection.query(
  query_embeddings=[target_embedding["embeddings"][0]],
  n_results=1
)
best_match = segments[int(query_result['ids'][0][0])]
'''

#difflib 匹配关键词
target = "推动经济全球化朝着更加开放、包容、普惠、平衡的方向发展"
best_match = None
best_score = 0

for item in segments:
    sim = SequenceMatcher(None, item['text'], target).ratio()
    item['sim'] = sim
    if sim > best_score:
        best_score = sim
        best_match = item

# 输出结果
print("🎯 匹配结果（按重组后句子）:")
for item in segments:
    print(f"文字：{item['text']}，相似度：{item['sim']:.2f}，坐标点数：{len(item['box'])}，坐标：{item['box']}")

print("\n🔥 最佳匹配:")
print(f"文本：{best_match['text']}，相似度：{best_match['sim']:.2f}，坐标：{best_match['box']}")

from PIL import Image, ImageDraw

# 加载原图（确保是RGBA）
base = Image.open("image_input.png").convert("RGBA")

# 创建一个透明图层
overlay = Image.new("RGBA", base.size, (0, 0, 0, 0))
draw = ImageDraw.Draw(overlay)

# 绘制半透明红色多边形到 overlay上
i = 0

while i < len(best_match['box']):
    draw.polygon(best_match['box'][i:i+4], fill=(255, 0, 0, 100))  # alpha=100 表示半透明
    i += 4

# 合成原图与染色图层
out = Image.alpha_composite(base, overlay)

# 保存最终效果
out.save("output_baidu.png")
print("✅ 染色结果已更新为半透明红色并保存为 output_baidu.png")


## 测试qwen系列能力

In [35]:
# 步骤 1：发出请求

from openai import OpenAI
import os
import json


question_path = "question.txt"
ground_truth_answer_path = "ground_truth_answer.txt"
explanation_of_ground_truth_answer_path = "explanation.txt"
few_shot_path = "few_shot.json"
student_answer_path = "student_answer.txt"

with open(f'{question_path}', 'r', encoding='utf-8') as question_file,\
    open(f'{ground_truth_answer_path}', 'r', encoding='utf-8') as ground_truth_answer_file,\
    open(f'{explanation_of_ground_truth_answer_path}', 'r', encoding='utf-8') as explanation_of_ground_truth_answer_file,\
    open(f'{few_shot_path}', 'r', encoding='utf-8') as few_shot_file,\
    open(f'{student_answer_path}', 'r', encoding='utf-8') as student_answer_file:
    question = question_file.read()  
    ground_truth_answer = ground_truth_answer_file.read()
    explanation_of_ground_truth_answer = explanation_of_ground_truth_answer_file.read()
    few_shot = json.load(few_shot_file)
    student_answer = student_answer_file.read()


client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    model="qwen3-32b",
    messages=[
        {
            "role": "system",
            "content": f"""现在你是一个中学老师，你要负责批改你学生的物理试卷的题目，
            主任要求你严格按照题目解析与各点评分标准里的踩分点来进行批改得分，
            并对每一个踩分点进行解释，比如说这个踩分点有对应的公式，得到相应的分数，
            那个踩分点没有公式或者公式错误，不得分"。按照批改示例里的“few_shot_output”的json格式输出
            题目：{question}
            标准答案：{ground_truth_answer}
            题目解析与各点评分标准：{explanation_of_ground_truth_answer}
            批改示例：{few_shot}
        """
        },
        {
            "role": "user",
            "content": f"学生答案：{student_answer}", 
        },
    ],
    response_format={"type": "json_object"},
    extra_body={"enable_thinking": False}
)

json_string = completion.choices[0].message.content
json_string = json.loads(json_string)
print(json_string)

{'points_earned_of_this_question': 4, 'correction_and_explanation': {'r_{乙} = \\frac{L}{\\sin 30^\\circ} = 2L': {'该点得分': 2, '原因': '存在该公式或其相同数学逻辑的形式，得2分'}, 'q v_0 B = m \\frac{v_0^2}{r_乙}': {'该点得分': 2, '原因': '存在该公式或其相同数学逻辑的形式，得2分'}, 'B = \\frac{m v_0}{2 q L}': {'该点得分': 0, '原因': '最终结果计算错误，正确表达式应为 $ B = \\frac{m v_0}{2 q L} $，而学生答案缺少了分母中的 $ L $，因此不得分'}}}


## 本地部署模型接管全流程

In [ ]:
# 步骤 1：发出请求

from openai import OpenAI
import os
import json


question_path = "question.txt"
ground_truth_answer_path = "ground_truth_answer.txt"
explanation_of_ground_truth_answer_path = "explanation.txt"
few_shot_path = "few_shot.json"
student_answer_path = "student_answer.txt"

with open(f'{question_path}', 'r', encoding='utf-8') as question_file,\
    open(f'{ground_truth_answer_path}', 'r', encoding='utf-8') as ground_truth_answer_file,\
    open(f'{explanation_of_ground_truth_answer_path}', 'r', encoding='utf-8') as explanation_of_ground_truth_answer_file,\
    open(f'{few_shot_path}', 'r', encoding='utf-8') as few_shot_file,\
    open(f'{student_answer_path}', 'r', encoding='utf-8') as student_answer_file:
    question = question_file.read()  
    ground_truth_answer = ground_truth_answer_file.read()
    explanation_of_ground_truth_answer = explanation_of_ground_truth_answer_file.read()
    few_shot = json.load(few_shot_file)
    student_answer = student_answer_file.read()


client = OpenAI(
    #api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="http://127.0.0.1:11434/v1",
)

completion = client.chat.completions.create(
    model="qwen3:8b",
    messages=[
        {
            "role": "system",
            "content": f"""现在你是一个中学老师，你要负责批改你学生的物理试卷的题目，
            主任要求你严格按照题目解析与各点评分标准里的踩分点来进行批改得分，
            并对每一个踩分点进行解释，比如说这个踩分点有对应的公式，得到相应的分数，
            那个踩分点没有公式或者公式错误，不得分"。按照批改示例里的“few_shot_output”的json格式输出
            题目：{question}
            标准答案：{ground_truth_answer}
            题目解析与各点评分标准：{explanation_of_ground_truth_answer}
            批改示例：{few_shot}
        """
        },
        {
            "role": "user",
            "content": f"学生答案：{student_answer}", 
        },
    ],
    response_format={"type": "json_object"},
    extra_body={"enable_thinking": False}
)

json_string = completion.choices[0].message.content
json_string = json.loads(json_string)
print(json_string)